## One Model

rosshandler: https://pubmed.ncbi.nlm.nih.gov/37982461/ \
nowotschin: https://pubmed.ncbi.nlm.nih.gov/30959515/ \
https://www.nature.com/articles/s41592-021-01336-8 \
https://docs.scvi-tools.org/en/stable/tutorials/notebooks/scrna/harmonization.html \
https://rapids-singlecell.readthedocs.io/en/latest/notebooks/demo_gpu-seuratv3-brain-1M.html 

In [3]:
%%time
# importing our library 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
#from scvi.models.utils import mde
import tempfile
import os
import scipy
from scipy.io import mmread
from scipy.io import mmwrite
import anndata as ad
import warnings 
import anndata as ad
import psutil
import h5py
import pickle
import sys

CPU times: user 4.94 s, sys: 1.3 s, total: 6.24 s
Wall time: 35.9 s


## Memory

In [16]:
## RAM info

# Get information about virtual memory usage
virtual_mem = psutil.virtual_memory()
# Access attributes like total, available, and used memory
total_mem = virtual_mem.total  # Total memory in bytes
available_mem = virtual_mem.available  # Available memory in bytes
used_mem = virtual_mem.used  # Used memory in bytes
# You can also calculate memory usage percentage
percent_used = used_mem / total_mem * 100

# printing it out
print(f"Total Memory: {total_mem / (1024**3)} GB")
print(f"Available Memory: {available_mem / (1024**3)} GB")
print(f"Used Memory: {used_mem / (1024**3)} GB")
print(f"Memory Usage Percentage: {percent_used:.2f}%")

Total Memory: 1007.155345916748 GB
Available Memory: 963.9385375976562 GB
Used Memory: 41.244651794433594 GB
Memory Usage Percentage: 4.10%


## Setting Path

In [17]:
data_path = '/data/hadjantalab/lucas/atlas/data'

## LOADING METADATA

In [18]:
%%time
## loading rosshandler
## setting the dtype for pandas to speed up loading of csv

file_path = os.path.join(data_path, "dask/adata_obs.csv")
adata_obs = pd.read_csv(
    file_path, 
    index_col = 'cell', 
    dtype={
        'cell': object,
        'sample': 'category', 
           'embryo_version': 'category', 
           'stage' : 'category', 
           'somite_count' : 'category', 
           'S_score' : np.float32, 
           'louvain' : 'category', 
           'leiden' : 'category', 
           'celltype_PijuanSala2019' : 'category', 
           'celltype_extended_atlas' : 'category', 
           'celltype_nowotschin' : 'category',
           'timepoint_nowotschin': 'category', 
           'cluster_nowotschin': 'category', 
           'batch': 'category'
          }, 
    engine = 'c')

file_path = os.path.join(data_path, "dask/adata_var.csv")
adata_var = pd.read_csv(
    file_path, 
    index_col = 0,
    dtype={0:object},
    engine = 'c')


CPU times: user 837 ms, sys: 29.6 ms, total: 867 ms
Wall time: 868 ms


<timed exec>:5: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.


## NEW NAMING CONVENTIONS COLUMNS ETC

In [19]:
#### all timepoints in one column

# Create sets for each column
stage_set = set(adata_obs['stage'])
timepoint_set = set(adata_obs['timepoint_nowotschin'])

# Combine the sets using union
combined_set = stage_set.union(timepoint_set)

# Convert the combined set to a list
categories = pd.Series(list(combined_set)).dropna()

# setting this categorical dtype
cat_type = pd.CategoricalDtype(categories=categories, ordered=True)

# setting same category types
adata_obs['stage'] = adata_obs['stage'].astype(cat_type)
adata_obs['timepoint_nowotschin'] = adata_obs['timepoint_nowotschin'].astype(cat_type)

# transfer nowotschin timepoints to rosshandler timepoints
adata_obs['stage'][adata_obs['batch'] == 'nowotschin'] = adata_obs['timepoint_nowotschin'][adata_obs['batch'] == 'nowotschin']
## removing _ap and _gfp from stage
adata_obs['stage'][(adata_obs['stage'] == 'E8.75_ap') | (adata_obs['stage'] == 'E8.75_gfp')] = 'E8.75'
## removing the category type
adata_obs['stage'] = adata_obs['stage'].cat.remove_categories(['E8.75_ap', 'E8.75_gfp'])

In [20]:
## getting highly variable gene info from rosshandler and nowotschin

hvg_rosshandler = pd.read_csv(
    '/data/hadjantalab/atlas/extAtlas/metadata_genes.csv', 
    index_col = 'mgi_symbol', 
    dtype = {'highly_variable' : 'boolean'},
    usecols = ['highly_variable', 'mgi_symbol'],
    engine = 'c')
hvg_rosshandler = hvg_rosshandler[~hvg_rosshandler.index.duplicated(keep='first')]


hvg_nowotschin = pd.read_csv(
    '/data/hadjantalab/atlas/endoderm/HVGs.csv', 
    index_col = 'index',
    engine = 'c')
hvg_nowotschin['mgi_symbol'] = hvg_nowotschin.index
hvg_nowotschin.index = hvg_nowotschin['mgi_symbol']
del hvg_nowotschin['mgi_symbol']

print(f'Number of hvg in rosshandler', hvg_rosshandler['highly_variable'].sum())
print(f'Number of hvg in nowotschin', len(hvg_nowotschin))

hvg = list(hvg_rosshandler[hvg_rosshandler['highly_variable']].index)
for i in hvg_nowotschin.index:
    if i not in hvg:
        hvg.append(i)
print(f'Number of hvg outer join {len(hvg)}')

Number of hvg in rosshandler 5997
Number of hvg in nowotschin 6000
Number of hvg outer join 8355


In [21]:
## adding highly variable gene info to our adata_var
adata_var['mgi_symbol'] = adata_var.index
adata_var.index = adata_var['mgi_symbol']
del adata_var['mgi_symbol']
adata_var['highly_variable'] = False
adata_var.index.isin(hvg)
adata_var['highly_variable'][adata_var.index.isin(hvg)] = True 

In [22]:
# TRYING TO GET SOME SAMPLE (SEQUENCING LANE) INFORMATION FOR THE NOWOTSCHIN DATASET

# Open the HDF5 file and read the data within the 'with' block
data_dict = {}  # Dictionary to store the data

with h5py.File('/data/hadjantalab/atlas/endoderm/manu_app_data/e8.5_metadata.h5', 'r') as f:
    # Get the first key in the file
    meta = list(f.keys())[0]
    # Access the group associated with the key
    meta_nowotschin = f[meta]

    if isinstance(meta_nowotschin, h5py.Group):
        print("This is a group. Keys in this group are:", list(meta_nowotschin.keys()))
        # Access and read all datasets within the group
        for key in meta_nowotschin.keys():
            dataset = meta_nowotschin[key]
            if isinstance(dataset, h5py.Dataset):
                data_dict[key] = dataset[...]
                print(f"Dataset {key} data: {data_dict[key]}")
    elif isinstance(meta_nowotschin, h5py.Dataset):
        # Read the dataset's data
        data_dict[meta] = meta_nowotschin[...]
        print("This is a dataset. Data is:", data_dict[meta])
    else:
        print("Unknown HDF5 object type.")

# Outside the 'with' block, you can now access 'data_dict' which contains the datasets' data
#print(data_dict)

### NO SEQUENCING LANE / SAMPLE INFO FOUND

This is a group. Keys in this group are: ['axis0', 'axis1', 'block0_items', 'block0_values', 'block1_items', 'block1_values']
Dataset axis0 data: [b'FDL_1' b'FDL_2' b'clusters' b'CellType' b'Timepoint']
Dataset axis1 data: [b'Lib7-1_E8.5_ParE_AAACCTGAGAGATGAG' b'Lib7-1_E8.5_ParE_AAACCTGAGGTGCTTT'
 b'Lib7-1_E8.5_ParE_AAACCTGAGTAGATGT' ...
 b'Lib8-2_E8.5_YsE_TTTGTCAGTACTTGAC' b'Lib8-2_E8.5_YsE_TTTGTCAGTCACCTAA'
 b'Lib8-2_E8.5_YsE_TTTGTCATCTTCGGTC']
Dataset block0_items data: [b'FDL_1' b'FDL_2']
Dataset block0_values data: [[  7.20748493  10.80574808]
 [ 24.68684971  -2.77087176]
 [  2.97991104  11.16478176]
 ...
 [-13.56749688 -12.30218697]
 [  5.69564038 -23.95019262]
 [ -9.92952847 -29.43028666]]
Dataset block1_items data: [b'clusters' b'CellType' b'Timepoint']
Dataset block1_values data: [array([128,   4, 149, ..., 148,  98,  46], dtype=uint8)]


In [23]:
## setting nowotschin dataset as celltype unknown for rosshandler annotation and vice versa
adata_obs['celltype_extended_atlas'] = adata_obs['celltype_extended_atlas'].cat.add_categories(['unknown'])
adata_obs['celltype_extended_atlas'][adata_obs['batch'] == 'nowotschin'] = 'unknown'
adata_obs['celltype_nowotschin'] = adata_obs['celltype_nowotschin'].cat.add_categories(['unknown'])
adata_obs['celltype_nowotschin'][adata_obs['batch'] != 'nowotschin'] = 'unknown'


In [24]:
## extracting sample info from the name of the nowotschin observations
nowo_sample_info = [x.rsplit('_', 1)[0] for x in adata_obs[adata_obs['batch'] == 'nowotschin'].index]
adata_obs['sample'] = adata_obs['sample'].cat.add_categories(set(nowo_sample_info))
## adding that information as sample info to adata_obs
adata_obs.loc[adata_obs['batch'] == 'nowotschin', 'sample'] = nowo_sample_info

In [25]:
### creating a compound column with the covariate batch, sample, and timepoint
adata_obs['covariate_composite'] = adata_obs['batch'].astype('string') + '_' + adata_obs['stage'].astype('string') + '_' + adata_obs['sample'].astype('string')

In [26]:
## deleting, renaming, and ordering

del adata_obs['embryo_version']
del adata_obs['timepoint_nowotschin']

adata_obs['celltype_rosshandler'] = adata_obs['celltype_extended_atlas'].copy()
del adata_obs['celltype_extended_atlas']

adata_obs['celltype_pijuansala'] = adata_obs['celltype_PijuanSala2019'].copy()
del adata_obs['celltype_PijuanSala2019']

adata_obs['rosshandler_somite_count'] = adata_obs['somite_count'].copy()
del adata_obs['somite_count']

adata_obs['rosshandler_anatomy'] = adata_obs['anatomy'].copy()
del adata_obs['anatomy']

adata_obs['rosshandler_S_score'] = adata_obs['S_score'].copy()
del adata_obs['S_score']

adata_obs['rosshandler_G2M_score'] = adata_obs['G2M_score'].copy()
del adata_obs['G2M_score']

adata_obs['rosshandler_phase'] = adata_obs['phase'].copy()
del adata_obs['phase']

adata_obs['rosshandler_louvain'] = adata_obs['louvain'].copy()
del adata_obs['louvain']

adata_obs['rosshandler_leiden'] = adata_obs['leiden'].copy()
del adata_obs['leiden']

adata_obs['nowotschin_cluster'] = adata_obs['cluster_nowotschin'].copy()
del adata_obs['cluster_nowotschin']


order = [ 'batch', 
         'stage', 
         'sample',
         'covariate_composite', 
         'celltype_nowotschin', 
         'celltype_rosshandler', 
         'celltype_pijuansala', 
         'nowotschin_cluster', 
         'rosshandler_somite_count', 
         'rosshandler_anatomy', 
         'rosshandler_phase',
         'rosshandler_S_score', 
         'rosshandler_G2M_score',
         'rosshandler_louvain', 
         'rosshandler_leiden']
adata_obs = adata_obs[order]

## RENAMING PROBLEMATIC ANNOTATION NAMES

In [28]:
adata_obs['celltype_rosshandler'] = adata_obs['celltype_rosshandler'].astype('object')
adata_obs.loc[adata_obs['celltype_rosshandler'] == 'NMPs/Mesoderm-biased', 'celltype_rosshandler'] = 'NMPs_Mesoderm-biased'
adata_obs.loc[adata_obs['celltype_rosshandler'] == 'Midbrain/Hindbrain boundary', 'celltype_rosshandler'] = 'Midbrain_Hindbrain boundary'

## LOADING COUNT TABLE

In [31]:
# function to read in our count matrix as csr_matrix
def load_sparse_csr(filename):
    loader = np.load(filename)
    return scipy.sparse.csr_matrix((loader['data'], loader['indices'], loader['indptr']), shape=loader['shape'])

In [32]:
%%time
file_path = os.path.join(data_path, "dask/adata_X.npz")

## reading in my csr_matrix
adata_x = load_sparse_csr(file_path)
print(f'X is a: {type(adata_x)} with the following shape: {adata_x.shape} and size: {(adata_x.data.nbytes + adata_x.indptr.nbytes + adata_x.indices.nbytes) /1e9} GB')

X is a: <class 'scipy.sparse._csr.csr_matrix'> with the following shape: (543390, 28652) and size: 40.211012984 GB
CPU times: user 34.5 s, sys: 11.5 s, total: 46 s
Wall time: 46.3 s


## ADATA SETUP 

In [33]:
# creating adata object
adata = ad.AnnData(X = adata_x, obs = adata_obs, var = adata_var)

In [34]:
# saving all the features (genes) in adata.raw
adata.raw = adata

In [35]:
# subsetting to our hvg
adata = adata[: ,adata.var['highly_variable']].copy()

In [36]:
## saving raw counts
adata.layers['raw_counts'] = adata.X.copy()

In [37]:
## getting some information on memory requirement 
print(f'X is a {type(adata.X)} with the following shape: {adata.X.shape} and size: {(adata.X.data.nbytes + adata.X.indptr.nbytes + adata.X.indices.nbytes) /1e9} GB')

X is a <class 'scipy.sparse._csr.csr_matrix'> with the following shape: (543390, 8355) and size: 6.10349782 GB


In [38]:
# getting some more information on memory requirement

def get_total_size_of_anndata(adata):
    total_size = sys.getsizeof(adata)  # Base size of the AnnData object
    
    # Add sizes of the attributes
    total_size += sys.getsizeof(adata.X)         # Expression matrix
    total_size += sys.getsizeof(adata.obs)       # Observations
    total_size += sys.getsizeof(adata.var)       # Variables (genes)
    total_size += sys.getsizeof(adata.obsm)      # Multi-dimensional observations
    total_size += sys.getsizeof(adata.varm)      # Multi-dimensional variables
    total_size += sys.getsizeof(adata.uns)       # Unstructured annotations
    total_size += sys.getsizeof(adata.layers)    # Layers of data
    
    # Include the size of adata.raw, if it exists
    if adata.raw is not None:
        total_size += sys.getsizeof(adata.raw)
        total_size += sys.getsizeof(adata.raw.X)
        total_size += sys.getsizeof(adata.raw.var)
    
    return total_size / (1024 ** 3)  # Convert bytes to GB

# Example usage:
print(f"Total size of AnnData object (in RAM): {get_total_size_of_anndata(adata):.2f} GB")

Total size of AnnData object (in RAM): 11.77 GB


## REMOVING 'MIXED_GASTRULATION' STAGE

In [39]:
mask = adata.obs['stage'] != 'Mixed gastrulation'
adata = adata[mask].copy()

## ADDING THE ANNOTATION

In [40]:

## creating new column
adata.obs['annotation'] = 'TO_BE_DETERMINED'
adata.obs.loc[adata.obs['batch'] == 'Sox17VEKO', 'annotation'] = 'To_be_determined'

## loading annotation table
data_path = '/data/hadjantalab/lucas/atlas/data'
file_path = os.path.join(data_path, 'annotation.csv')
annotation = pd.read_table(file_path, sep=',', index_col='cell')

## Converting annotation DataFrame to a dictionary with direct cell-to-annotation mapping
annotation = annotation.loc[:, 'annotation'].to_dict()

## Setting values in adata according to the annotation table
adata.obs.loc[adata.obs.index.isin(annotation.keys()), 'annotation'] = adata.obs.index.map(annotation).dropna()


In [ ]:
adata.obs['annotation'] = adata.obs['annotation'].astype('object')
adata.obs.loc[adata.obs['annotation'] == 'NMPs/Mesoderm-biased', 'annotation'] = 'NMPs_Mesoderm-biased'
adata.obs.loc[adata.obs['annotation'] == 'Midbrain/Hindbrain boundary', 'annotation'] = 'Midbrain_Hindbrain boundary'

## CREATING 10E6 LOG2 NORMALIZATION

In [10]:
adata.X = adata.layers['raw_counts'].copy()

In [11]:
np.set_printoptions(threshold=np.inf)

subset = adata.X[10000:10050, 5000:5050].toarray()
print(subset)

np.set_printoptions(threshold=1000)  # or use np.set_printoptions() to reset to default


[[ 0.  7.  0.  1.  0.  3. 30.  0.  0.  0.  0.  0.  0.  8.  4.  0.  0.  0.
   0.  2.  0.  0.  2.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
   0.  0. 27. 18.  2.  1. 10.  4.  5.  1.  3.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  2. 16.  0.  0.  0.  0.  0.  0.  4.  4.  0.  0.  0.
   0.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
   0.  0.  7.  6.  1.  1.  4.  3.  5.  1.  6.  0.  0.  0.]
 [ 0.  1.  0.  1.  0.  5.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
   0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  4.  3.  0.  0.  1.  2.  3.  0.  4.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.  2. 22.  1.  0.  0.  0.  0.  0.  6.  4.  0.  1.  0.
   0.  1.  0.  0.  0.  1.  0.  0.  2.  0.  0.  0.  0.  0.  1.  0.  0.  0.
   0.  0. 16. 13.  3.  1.  8.  2.  4.  0.  5.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  3.  6.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  1.  4.  0.  0

In [12]:
sc.pp.normalize_total(adata, target_sum=1e4)


In [13]:
np.set_printoptions(threshold=np.inf)

subset = adata.X[10000:10050, 5000:5050].toarray()
print(subset)

np.set_printoptions(threshold=1000)  # or use np.set_printoptions() to reset to default


[[ 0.          8.01282     0.          1.1446886   0.          3.4340658
  34.34066     0.          0.          0.          0.          0.
   0.          9.157509    4.5787544   0.          0.          0.
   0.          2.2893772   0.          0.          2.2893772   0.
   0.          0.          0.          0.          0.          0.
   1.1446886   0.          0.          0.          0.          0.
   0.          0.         30.906593   20.604395    2.2893772   1.1446886
  11.446886    4.5787544   5.723443    1.1446886   3.4340658   0.
   0.          0.        ]
 [ 0.          1.9677293   1.9677293   0.          0.          3.9354587
  31.48367     0.          0.          0.          0.          0.
   0.          7.8709173   7.8709173   0.          0.          0.
   0.          3.9354587   0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          1.9677293   0.          0.          0.
   0.          0.         13

In [14]:
sc.pp.log1p(adata, base = 2)


In [15]:
np.set_printoptions(threshold=np.inf)

subset = adata.X[10000:10050, 5000:5050].toarray()
print(subset)

np.set_printoptions(threshold=1000)  # or use np.set_printoptions() to reset to default


[[0.         3.1719785  0.         1.1007682  0.         2.1486301
  5.143257   0.         0.         0.         0.         0.
  0.         3.3444748  2.479943   0.         0.         0.
  0.         1.7178144  0.         0.         1.7178144  0.
  0.         0.         0.         0.         0.         0.
  1.1007682  0.         0.         0.         0.         0.
  0.         0.         4.995783   4.433253   1.7178144  1.1007682
  3.637713   2.479943   2.7492003  1.1007682  2.1486301  0.
  0.         0.        ]
 [0.         1.5693595  1.5693595  0.         0.         2.303184
  5.0216427  0.         0.         0.         0.         0.
  0.         3.1490831  3.1490831  0.         0.         0.
  0.         2.303184   0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         1.5693595  0.         0.         0.
  0.         0.         3.884999   3.6787903  1.5693595  1.5693595
  3.1490831  2.7872627  3.4381127  1.5693595  3.

In [16]:
adata.layers['log_normalized'] = adata.X.copy()

In [17]:
## setting adata.X back to raw counts
adata.X = adata.layers['raw_counts'].copy()

## Save data 

In [45]:
#%%time
## saving everything seperately

## all metadata
#file_path = os.path.join(data_path, "adata_obs.csv")
#adata.obs.to_csv(file_path, index = 'cell')

#file_path = os.path.join(data_path, "adata_var.csv")
#adata.var.to_csv(file_path, index = 'mgi_symbol')

#def save_sparse_csr(filename, array):
#    np.savez(filename, data=array.data, indices=array.indices, indptr=array.indptr, shape=array.shape)

## saving sparse matrix seperately
#file_path = os.path.join(data_path, "adata_X.npz")
#save_sparse_csr(file_path, adata.X)

In [18]:
data_path

'/data/hadjantalab/lucas/atlas/data'

In [19]:
%%time
## saving the complete object, it ain't that big
file_path = os.path.join(data_path, "adata.h5ad")
adata.write_h5ad(file_path)

CPU times: user 14.9 s, sys: 17.8 s, total: 32.7 s
Wall time: 47.4 s


## LOAD DATA

In [20]:
data_path = '/data/hadjantalab/lucas/atlas/data'

In [21]:
%%time

file_path = os.path.join(data_path, "adata.h5ad")
adata = ad.read_h5ad(file_path)

CPU times: user 14.6 s, sys: 45.2 s, total: 59.9 s
Wall time: 1min 9s
